In [2]:
import csv

In [3]:
import pandas as pd
import sqlite3


In [ ]:
# Load the order CSV 
orders_df = pd.read_csv("orders.csv")

#To Preview data
orders_df.head()
# Load Users JSON 
users_df = pd.read_json("users.json")

# Preview data
users_df.head()



,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
# Create SQLite database connection
conn = sqlite3.connect("restaurants.db")

# Read SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

# Execute SQL script
conn.executescript(sql_script)

# Load restaurants table into DataFrame
restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)

# Preview data
restaurants_df.head()



,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [6]:
# Merge orders with users on user_id
orders_users_df = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)

#Here we Preview merged data
orders_users_df.head()

# Merge previous result with restaurants on restaurant_id
final_df = pd.merge(
    orders_users_df,
    restaurants_df,
    on="restaurant_id",
    how="left"
)

# Preview final dataset
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
# Save final dataset as CSV
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("✅ Final dataset saved as : final_food_delivery_dataset.csv")


✅ Final dataset saved as : final_food_delivery_dataset.csv


In [9]:
df = pd.read_csv("final_food_delivery_dataset.csv")
df.head()
# loading the final csv file
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue



city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [10]:
gold_city_avg = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_avg


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [11]:
cuisine_avg = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

cuisine_avg


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [14]:
user_spend = (
    df.groupby("user_id")["total_amount"].sum())

count_users = user_spend[user_spend > 1000].count()
count_users


2544

In [15]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)
rating_revenue = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
rating_revenue


C:\Users\adesh\AppData\Local\Temp\ipykernel_4856\1535687811.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [16]:

gold_city_avg = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_avg


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [17]:
cuisine_summary = (
    df.groupby("cuisine")
    .agg(
        restaurant_count=("restaurant_id", "nunique"),
        revenue=("total_amount", "sum")
    )
    .sort_values("restaurant_count")
)

cuisine_summary


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [18]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

percentage = round((gold_orders / total_orders) * 100)
percentage


50

In [19]:
restaurant_stats = (
    df.groupby("restaurant_name_y")
    .agg(
        avg_order=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
)

filtered = restaurant_stats[
    restaurant_stats["total_orders"] < 20
].sort_values("avg_order", ascending=False)

filtered.head()


,avg_order,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16


In [20]:
combo_revenue = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
combo_revenue


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [22]:
df["quarter"] = df["order_date"].dt.quarter

quarter_revenue = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
quarter_revenue

AttributeError: Can only use .dt accessor with datetimelike values

In [23]:
# Clean duplicate header if present
df = df[df["order_id"] != "order_id"]
df.reset_index(drop=True, inplace=True)


In [ ]:
gold_orders_count = df[df["membership"] == "Gold"].shape[0] #count orders using .shape to get size of the first dimension
gold_orders_count

4987

In [25]:
hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)
hyderabad_revenue

1889367

In [26]:
distinct_users = df["user_id"].nunique() #nunique returns a the total number of distinct elements 
distinct_users

2883

In [27]:
gold_avg_order_value = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
    2 )
gold_avg_order_value

797.15

In [28]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
high_rating_orders

3374

In [30]:
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax() )
top_gold_city

orders_in_top_city = (
    df[(df["membership"] == "Gold") & (df["city"] == top_gold_city)]
    .shape[0]
)
orders_in_top_city

1337